In [3]:
import librosa
print(librosa.__file__)


FileNotFoundError: Could not find module 'C:\Users\USER\anaconda3\envs\new_env\lib\site-packages\scipy\.libs\libbanded5x.2KADQYTNPWKJMTCXD6RBF57D5WTRNEE3.gfortran-win_amd64.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [1]:
import os
import time
from pydub import AudioSegment
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

# mp3 파일이 있는 디렉토리 경로
input_root_directory = "./data/Training/"
# 멜 스펙트로그렘 데이터를 저장할 디렉토리 경로
mel_output_directory = './data/result_wav/'

# 잘라낼 앞부분 시간 (밀리초 단위)
trim_time = 10 * 1000  # 10초

# 결과 폴더가 없으면 생성
if not os.path.exists(mel_output_directory):
    os.makedirs(mel_output_directory)

# 폴더 인덱스 초기화
folder_index = 0

def save_mel_spectrogram(chunk_data, sr, filename):
    S = librosa.feature.melspectrogram(y=chunk_data, sr=sr, n_mels=128)
    S_DB = librosa.power_to_db(S, ref=np.max)
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(S_DB, sr=sr, hop_length=512)
    plt.tight_layout()
    plt.savefig(os.path.join(mel_output_directory, filename))
    plt.close()

# 입력 디렉토리 내 모든 파일을 확인
for root, dirs, files in os.walk(input_root_directory):
    d_name = os.path.basename(os.path.normpath(root))
    if root == input_root_directory:  # 루트 디렉토리 자체는 건너뛴다
        continue

    # 각 폴더에 대한 카운터 초기화
    i = 0
    for filename in files:
        if filename.endswith(".mp3") or filename.endswith(".wav"):
            # mp3, wav 파일의 전체 경로
            file_path = os.path.join(root, filename)
            
            start_time = time.time()
            
            try:
                # 파일을 AudioSegment 객체로 불러오기
                if filename.endswith(".mp3"):
                    audio = AudioSegment.from_file(file_path, format="mp3")
                    # 앞부분 10초를 잘라내기
                    audio = audio[trim_time:]
                elif filename.endswith(".wav"):
                    audio = AudioSegment.from_file(file_path, format="wav")

                # AudioSegment를 numpy 배열로 변환
                samples = np.array(audio.get_array_of_samples(), dtype=np.float32)
                sr = audio.frame_rate

                # 모노로 변환
                if audio.channels > 1:
                    samples = librosa.to_mono(samples.T)

                # 전체 데이터 길이 계산
                duration = len(samples) / sr

                # 데이터 길이에 따른 처리
                if duration < 4:
                    print(f"파일 {filename}는 4초 미만으로 처리되지 않습니다.")
                elif 6 <= duration <= 8:
                    for chunk_index in range(2):
                        start_sample = chunk_index * (duration - 4) * sr
                        end_sample = start_sample + 4 * sr
                        chunk_data = samples[int(start_sample):int(end_sample)]
                        mel_filename = f"{folder_index}_{d_name}_{i}.png"
                        save_mel_spectrogram(chunk_data, sr, mel_filename)
                        end_time = time.time()
                        elapsed_time = end_time - start_time
                        print(f"변환 완료: {filename} -> {mel_filename} (소요 시간: {elapsed_time:.2f}초)")
                        i += 1
                        
                else:
                    num_chunks = int(duration // 4)
                    for chunk_index in range(num_chunks):
                        start_sample = chunk_index * 4 * sr
                        end_sample = min((chunk_index + 1) * 4 * sr, len(samples))
                        chunk_data = samples[int(start_sample):int(end_sample)]
                        mel_filename = f"{folder_index}_{d_name}_{i}.png"
                        save_mel_spectrogram(chunk_data, sr, mel_filename)
                        end_time = time.time()
                        elapsed_time = end_time - start_time
                        print(f"변환 완료: {filename} -> {mel_filename} (소요 시간: {elapsed_time:.2f}초)")
                        i += 1
                        
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
    
    # 폴더 인덱스 증가
    folder_index += 1

print("모든 변환이 완료되었습니다!")


변환 완료: 02c3b725-26e4-4a2c-9336-04ddc58836d9-1430726196216-1.7-m-04-hu.wav -> 0_babycry_0.png (소요 시간: 0.10초)
변환 완료: 02c3b725-26e4-4a2c-9336-04ddc58836d9-1430726196216-1.7-m-04-hu.wav -> 0_babycry_1.png (소요 시간: 0.15초)
변환 완료: 02ead89b-aa02-453e-8b83-6ebde9fe7551-1430233132879-1.7-m-26-hu.wav -> 0_babycry_2.png (소요 시간: 0.05초)
변환 완료: 02ead89b-aa02-453e-8b83-6ebde9fe7551-1430233132879-1.7-m-26-hu.wav -> 0_babycry_3.png (소요 시간: 0.09초)
변환 완료: 035c6b30-a145-42b9-8d0f-445cd9003d2c-1435948197257-1.7-m-04-hu.wav -> 0_babycry_4.png (소요 시간: 0.05초)
변환 완료: 035c6b30-a145-42b9-8d0f-445cd9003d2c-1435948197257-1.7-m-04-hu.wav -> 0_babycry_5.png (소요 시간: 0.09초)
변환 완료: 03abcb8f-400a-47d8-ad82-7e4586cc06be-1431864192133-1.7-f-48-hu.wav -> 0_babycry_6.png (소요 시간: 0.06초)
변환 완료: 03abcb8f-400a-47d8-ad82-7e4586cc06be-1431864192133-1.7-f-48-hu.wav -> 0_babycry_7.png (소요 시간: 0.11초)
변환 완료: 03ADDCFB-354E-416D-BF32-260CF47F7060-1433658024-1.1-f-04-ti.wav -> 0_babycry_8.png (소요 시간: 0.06초)
변환 완료: 03ADDCFB-354E-416D-BF32-


KeyboardInterrupt



<Figure size 1000x400 with 0 Axes>

In [1]:
import os
import time
from pydub import AudioSegment
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

# mp3 파일이 있는 디렉토리 경로
input_root_directory = "./data/Training/"
# 멜 스펙트로그렘 데이터를 저장할 디렉토리 경로
mel_output_directory = './data/result_wav/'
# 변환된 WAV 파일을 저장할 임시 디렉토리 경로
temp_folder = './data/temp/'

# 잘라낼 앞부분 시간 (밀리초 단위)
trim_time = 10 * 1000  # 10초

# 결과 폴더가 없으면 생성
if not os.path.exists(mel_output_directory):
    os.makedirs(mel_output_directory)
if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)

# 멜 스펙트로그렘 이미지를 저장하는 함수
def save_mel_spectrogram(chunk_data, sr, filename):
    S = librosa.feature.melspectrogram(y=chunk_data, sr=sr, n_mels=128)
    S_DB = librosa.power_to_db(S, ref=np.max)
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(S_DB, sr=sr, hop_length=512)
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

# 입력 디렉토리 내 모든 파일을 확인
folder_index = 6
for root, dirs, files in os.walk(input_root_directory):
    d_name = os.path.basename(os.path.normpath(root))
    if root == input_root_directory:  # 루트 디렉토리 자체는 건너뛴다
        continue

    # 각 폴더에 대한 카운터 초기화
    file_index = 0
    for filename in files:
        if filename.endswith(".mp3") or filename.endswith(".wav"):
            # mp3, wav 파일의 전체 경로
            file_path = os.path.join(root, filename)
            
            start_time = time.time()
            
            try:
                # 파일을 AudioSegment 객체로 불러오기
                if filename.endswith(".mp3"):
                    audio = AudioSegment.from_file(file_path, format="mp3")
                    # 앞부분 10초를 잘라내기
                    audio = audio[trim_time:]
                    # 임시 파일 경로 설정
                    temp_wav_path = os.path.join(temp_folder, filename.replace(".mp3", ".wav"))
                    audio.export(temp_wav_path, format="wav")
                    file_path = temp_wav_path
                elif filename.endswith(".wav"):
                    audio = AudioSegment.from_file(file_path, format="wav")
                
                # AudioSegment를 numpy 배열로 변환
                samples = np.array(audio.get_array_of_samples(), dtype=np.float32)
                sr = audio.frame_rate

                # 모노로 변환
                if audio.channels > 1:
                    samples = librosa.to_mono(samples.T)

                # 전체 데이터 길이 계산
                duration = len(samples) / sr

                # 데이터 길이에 따른 처리
                if duration < 4:
                    print(f"파일 {filename}는 4초 미만으로 처리되지 않습니다.")
                else:
                    num_chunks = int(duration // 4)
                    for chunk_index in range(num_chunks):
                        start_sample = chunk_index * 4 * sr
                        end_sample = min((chunk_index + 1) * 4 * sr, len(samples))
                        chunk_data = samples[int(start_sample):int(end_sample)]
                        mel_filename = os.path.join(mel_output_directory, f"{folder_index}_{d_name}_{file_index}.png")
                        save_mel_spectrogram(chunk_data, sr, mel_filename)
                        end_time = time.time()
                        elapsed_time = end_time - start_time
                        print(f"변환 완료: {filename} -> {mel_filename} (소요 시간: {elapsed_time:.2f}초)")
                        file_index += 1

            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

    folder_index += 1

print("모든 변환이 완료되었습니다!")


변환 완료: S-211031_L_101_C_035_0001.mp3 -> ./data/result_wav/6_gas_0.png (소요 시간: 0.24초)
변환 완료: S-211031_L_101_C_035_0001.mp3 -> ./data/result_wav/6_gas_1.png (소요 시간: 0.37초)
변환 완료: S-211031_L_101_C_035_0001.mp3 -> ./data/result_wav/6_gas_2.png (소요 시간: 0.46초)
변환 완료: S-211031_L_101_C_035_0001.mp3 -> ./data/result_wav/6_gas_3.png (소요 시간: 0.56초)
변환 완료: S-211031_L_101_C_035_0001.mp3 -> ./data/result_wav/6_gas_4.png (소요 시간: 0.65초)
변환 완료: S-211031_L_101_C_035_0001.mp3 -> ./data/result_wav/6_gas_5.png (소요 시간: 0.74초)
변환 완료: S-211031_L_101_C_036_0001.mp3 -> ./data/result_wav/6_gas_6.png (소요 시간: 0.19초)
변환 완료: S-211031_L_101_C_036_0001.mp3 -> ./data/result_wav/6_gas_7.png (소요 시간: 0.29초)
변환 완료: S-211031_L_101_C_036_0001.mp3 -> ./data/result_wav/6_gas_8.png (소요 시간: 0.39초)
변환 완료: S-211031_L_101_C_036_0001.mp3 -> ./data/result_wav/6_gas_9.png (소요 시간: 0.50초)
변환 완료: S-211031_L_101_C_036_0001.mp3 -> ./data/result_wav/6_gas_10.png (소요 시간: 0.61초)
변환 완료: S-211031_L_101_C_036_0001.mp3 -> ./data/result_wav/6_gas_